# Importación de módulos


In [ ]:
!pip install pdfminer

import pdfminer

from io import StringIO

from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.layout import LAParams

!pip install --upgrade spacy

import spacy
from spacy import displacy

!python -m spacy download es_core_news_md
!python -m spacy download de_core_news_md

nlp_es = spacy.load("es_core_news_md")
nlp_de = spacy.load("de_core_news_md")

from spacy.matcher import Matcher
matcher = Matcher(nlp_de.vocab)

import collections
import string
import csv
import matplotlib
import pandas
import re
import pprint
import numpy as np
import matplotlib.pyplot as plt
from pandas import DataFrame

from google.colab import files

punct=string.punctuation

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2022-10-13 18:19:30.499256: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://github.com/explosion/spacy-models/releases/download/es_core_news_md-3.4.0/es_core_news_md-3.4.0-py3-none-any.whl (42.3 MB)
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')
2022-10-13 18:19:40.716260: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://github.co

# Localización de textos a partir de una cadena

In [ ]:
textos_pdf= [i for i in range(22,25)]+[i for i in range(26,35)] #textos de 1826 

In [ ]:
archivo=!unzip Archiv.zip

In [ ]:
print([i for i in archivo])

['Archive:  Archiv.zip', '  End-of-central-directory signature not found.  Either this file is not', '  a zipfile, or it constitutes one disk of a multi-part archive.  In the', '  latter case the central directory and zipfile comment will be found on', '  the last disk(s) of this archive.', 'unzip:  cannot find zipfile directory in one of Archiv.zip or', '        Archiv.zip.zip, and cannot find Archiv.zip.ZIP, period.']


In [ ]:
def buscador_toponimos(textos,toponimo): #declaración de la función
    
    for i in textos: #itera "textos" y recupera cada número en la lista
        with open(str(i)+".pdf", 'rb') as file: #abre cada documento con los números de la lista como nombre
            rsrcmgr = PDFResourceManager() #esta línea y las siguientes son las configuraciones necesarias para extraer el texto con pdfminer
            retstr = StringIO()
            device = TextConverter(rsrcmgr, retstr, codec="utf-8", laparams=LAParams())
            interpreter = PDFPageInterpreter(rsrcmgr, device)

            pages_text = [] #aquí se almacena el texto página por página
            page_count=0 #contador de páginas del PDF

            num_texto=str(i) #recupera el nombre (que es igual al número) del texto

            for page in PDFPage.get_pages(file): #esta línea y las siguientes son las configuraciones necesarias para almacenar el texto, página por página, en la variable pages_text 
                read_position = retstr.tell()
                interpreter.process_page(page)
                retstr.seek(read_position, 0)
                page_text = retstr.read()
                pages_text.append(page_text)

        for i in pages_text: #una vez almacenado el texto de cada página en listas se itera cada lista
            page_count+=1 #se actualiza el contador de páginas por cada iteración
            lista_palabras=i.split() #se separa en palabras la cadena de la página
            lista_top=[j for j in lista_palabras if toponimo in j] #se almacenan las palabras que coincidan con la cadena de caracteres ingresada al llamar a la función en la variable lista_top
            if len(lista_top)>0: #se comprueba que la lista no esté vacía
                print("Texto: "+str(num_texto)+" ,página: "+str(page_count)+":") #se imprime encabezado con el numero de texto y de pá
                print(lista_top) #se imprimen los resultados
                print("\n")

In [ ]:
buscador_toponimos(textos_pdf,"Mexik")

#Problemas:
#Es necesario ingresar cada variante del topónimo

# Localización de "México" con Regex y SpaCy

In [ ]:
for i in textos_pdf:

    with open("/Users/erickvillanueva/Documents/Textos Corpus De - Editado/"+str(i)+".pdf", 'rb') as file:

        rsrcmgr = PDFResourceManager()
        retstr = StringIO()
        device = TextConverter(rsrcmgr, retstr, codec="utf-8", laparams=LAParams())
        interpreter = PDFPageInterpreter(rsrcmgr, device)

        pages_text = []
        page_count=0

        num_texto=str(i)

        for page in PDFPage.get_pages(file):
            read_position = retstr.tell()
            interpreter.process_page(page)
            retstr.seek(read_position, 0)
            page_text = retstr.read()
            pages_text.append(page_text)
    
    for i in pages_text:
        page_count+=1
        matches = re.finditer(r"(M|m)e(x|j)i(c|k)\S+", i) #se crea una tupla con los matches de la expresión regular; (x|y): alternancia entre caracteres; \w+: 1 o más caracteres alfanuméricos 
        
        lista_matches=[k.group() for k in matches] # se crea una lista de la tupla para poder iterarla

        nom1 = [{"LOWER": "neu spanien"}] #se crea un patrón de Matcher para "Neu Spanien" con cualquier caracter entre ambos elementos; , borré: {...},{"IS_SPACE":True}, {"LOWER": "spanien"}
        nom2 = [{"LOWER": "neuspanien"}] # otro patrón para "Neuspanien"
        nom3 = [{"LOWER": "neu-spanien"}] # otro patrón para "Neu-Spanien"

        matcher.add("Neuspanien", [nom1,nom2,nom3]) #se crea un modelo con los patrones

        for match_id, start, end in matcher(nlp_de(i)): #se crea una cadena de la lista de matches anterior como objeto de SpaCy; se llama los matches y sus "coordenadas" 
            string_id = nlp_de.vocab.strings[match_id] #se extrae la cadena del objeto de SpaCy  
            span = nlp_de(i)[start:end] #se guardan las coordenadas token en span                 
            lista_matches.append(span.text) # se añaden los matches a lista_matches

        if lista_matches:
            print("Texto: "+str(num_texto)+" ,página: "+str(page_count)+":")
            print(lista_matches)
            print("\n")

    print("–––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––")

#Problemas:
#Es necesario conocer las alternancias de grafia y las variantes nominales de diversidad de palabras e ingresarlas para cada topónimo

In [ ]:
#OBTENER NUMERO DE PALABRAS DE CADA TOPÓNIMO
#RELACIONADO CON EL CÁLCULO DE VERDADEROS NEGATIVOS PARA EL F1-SCORE
#COMPARÉ LOS RESULTADOS CON EL CONTADOR DE PALABRAS DE WORD Y HAY COINCIDENCIA CASI PERFECTA

for i in textos_pdf:

    with open("/Users/erickvillanueva/Documents/Textos Corpus De - Editado/"+str(i)+".pdf", 'rb') as file:

        rsrcmgr = PDFResourceManager()
        retstr = StringIO()
        device = TextConverter(rsrcmgr, retstr, codec="utf-8", laparams=LAParams())
        interpreter = PDFPageInterpreter(rsrcmgr, device)

        pages_text = []
        page_count=0
        
        conteo=0
        
        num_texto=str(i)

        for page in PDFPage.get_pages(file):
            read_position = retstr.tell()
            interpreter.process_page(page)
            retstr.seek(read_position, 0)
            page_text = retstr.read()
            pages_text.append(page_text)
      
        for i in pages_text:
            conteo+=len(i.split())
    
        
    print("Texto: "+str(num_texto)+": "+str(conteo))

In [ ]:
#OBTENER TEXTO POR PÁGINA – PARA COTEJAR CON LOS ARCHIVOS CVS (EL GOLD STANDARD)

for i in range(24,25):

    with open("/Users/erickvillanueva/Documents/Textos Corpus De - Editado/"+str(i)+".pdf", 'rb') as file:

        rsrcmgr = PDFResourceManager()
        retstr = StringIO()
        device = TextConverter(rsrcmgr, retstr, codec="utf-8", laparams=LAParams())
        interpreter = PDFPageInterpreter(rsrcmgr, device)

        pages_text = []
        page_count=0
        
        conteo=0
        
        num_texto=str(i)

        for page in PDFPage.get_pages(file):
            read_position = retstr.tell()
            interpreter.process_page(page)
            retstr.seek(read_position, 0)
            page_text = retstr.read()
            pages_text.append(page_text)
      
        for i in pages_text:
            page_count+=1 
            print("Texto: "+str(num_texto)+", página: "+str(page_count)+" –––––––––––––––––––––––––––––––––––––––––––")
            print("\n")
            print(i)
            print("\n")
        
    print("–––––––––––––––––––––––– Fin del texto ––––––––––––––––––––––––")